imports

In [1]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath('..'))


In [2]:
import numpy as np
import pandas as pd

import src.preprocessing as pre
import src.utils as utils
import src.table as table
import src.graphs as graphs

initialize shit

In [3]:
train, test, mappings, meta = pre.separate_year('../data/meta.json','../data/y.csv','../data/mappings.json')
model = utils.create_model(train,num_epochs=1)
preds = utils.predict_gridded_harvest(model,test)
actuals = test.Y.detach().numpy()
production_plan = pre.decode(meta,mappings).reset_index(drop=True)
idx_dict = table.create_indices_dict(production_plan)

transplant_weeks = production_plan.WeekTransplanted.to_numpy()
year_preds, year_actuals = table.season_shift(transplant_weeks,preds,actuals)
year_preds.shape,year_actuals.shape

Epoch [1/200], Loss: 4380361.5384


((139, 20, 51), (139, 51))

In [4]:
preds.shape, actuals.shape

((139, 20, 20), (139, 20))

full season totals

In [5]:
fig1 = graphs.graph_harvest(year_preds,year_actuals)

In [6]:
fig1.show()

received only harvest cycle

In [7]:
nonzero_indices = np.any(actuals != 0, axis=1)
actuals_received = actuals[nonzero_indices]
preds_received = preds[nonzero_indices]

fig2 = graphs.graph_harvest(preds_received,actuals_received)

In [8]:
actuals_received.shape

(139, 20)

In [9]:
fig4 = graphs.graph_harvest_cumsum(preds_received,actuals_received)

In [10]:
fig2.show()

In [11]:
fig4.show()

total harvest cycle

In [24]:
fig3 = graphs.graph_harvest(preds,actuals)

In [27]:
fig5 = graphs.graph_harvest_cumsum(preds,actuals)

In [25]:
fig3.show()

In [28]:
fig5.show()

In [15]:
summary_class = production_plan.groupby(['Class','Type']).agg(
    {'Ha':'sum',
     'WeekTransplanted':['min','max','nunique'],
     'Ranch':'nunique'
})

In [16]:
types = summary_class.index.get_level_values('Type')

In [17]:
preds_all_types, actuals_all_types = table.filter_preds(preds,actuals,'Type',types,idx_dict)
preds_all_types.shape,actuals_all_types.shape

((7, 20, 20), (7, 20))

In [18]:
actuals_received = actuals_all_types.sum(axis=1)
final_preds = preds_all_types.sum(axis=2)[:,-1]
initial_preds = preds_all_types.sum(axis=2)[:,0]

table summary, important stats, % expected, amount received, adjusted %, initial predictions, final predictions, totals by class, tomato type

In [19]:
summary_class['KilosReceived'] = actuals_received
summary_class['FinalPreds'] = final_preds
summary_class['InitialPreds'] = initial_preds
summary_class

Ha WeekTransplanted               Ranch  \
                          sum              min max nunique nunique   
Class Type                                                           
BSUF  Uva Amarilla     2.4745                6  28      12       6   
      Uva Roja        27.9135                4  31      19       6   
CHE   Cherry Naranja   0.6529               12  30       9       5   
      Cherry Rojo     18.2069                4  31      15       6   
      Cherry Rosa      3.7839                9  29      13       6   
      Pera Amarilla    0.6037               14  24       6       6   
      Pera Roja        0.4607               11  24       5       4   

                      KilosReceived     FinalPreds InitialPreds  
                                                                 
Class Type                                                       
BSUF  Uva Amarilla     26024.000000   24571.696404          0.0  
      Uva Roja        678726.062500  615182.192646          0.0  
CHE   Cherry Naranja    7677.000000    8057.297036          0.0  
      Cherry Rojo     429096.031250  379326.879144          0.0  
      Cherry Rosa      48863.000000   44411.496024          0.0  
      Pera Amarilla     6097.999512    6664.046774          0.0  
      Pera Roja         3880.000000    4146.844033          0.0

by class

In [33]:
classes = ['BSUF','CHE']
classes_idx = table.get_indices('Class',classes,idx_dict)
class_transplant_weeks = transplant_weeks[classes_idx]

In [56]:
year_preds_by_class, year_actuals_by_class = table.filter_preds(year_preds,year_actuals,'Class',classes,idx_dict)
preds_by_class, actuals_by_class = table.filter_preds(preds,actuals,'Class',classes,idx_dict)

In [57]:
fig6 = graphs.graph_harvest_stacked(year_preds_by_class,year_actuals_by_class,classes)
fig6b = graphs.graph_harvest_stacked(preds_by_class,actuals_by_class,classes)
fig6.show()
fig6b.show()

by type

In [48]:
types = ['Uva Roja','Cherry Rosa']
types_idx = table.get_indices('Type',types,idx_dict)
type_transplant_weeks = transplant_weeks[types_idx]


In [49]:
year_preds_by_type, year_actuals_by_type = table.filter_preds(year_preds,year_actuals,'Type',types,idx_dict)

In [51]:
fig7 = graphs.graph_harvest_stacked(year_preds_by_class,year_actuals_by_class,classes)
fig7.show()

by ranch, all 3 and table

In [52]:
ranches = ['OAP','VAP']
ranches_idx = table.get_indices('Ranch',ranches,idx_dict)
ranch_transplant_weeks = transplant_weeks[ranches_idx]

In [53]:
year_preds_by_ranch, year_actuals_by_ranch = table.filter_preds(year_preds,year_actuals,'Ranch',ranches,idx_dict)

In [55]:
fig8 = graphs.graph_harvest_stacked(year_preds_by_ranch,year_actuals_by_ranch,ranches)
fig8.show()